[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/shhommychon/DeZero-Koki/blob/master/from_scratch_3/step07.ipynb)

# 제1고지 미분 자동 계산

## 1단계 상자로서의 변수

## 2단계 변수를 낳는 함수

## 3단계 함수 연결

## 4단계 수치 미분

In [1]:
# def numerical_diff(f, x, eps=1e-4):
#     """

#       param:
#         f (Function): 미분의 대상이 되는 함수
#         x (Variable): 미분을 계산하는 변수
#         eps (float): 작은 값
#     """
#     x0 = Variable(x.data - eps)
#     x1 = Variable(x.data + eps)
#     y0 = f(x0)
#     y1 = f(x1)
#     return (y1.data - y0.data) / (2 * eps)

## 5단계 역전파 이론

## 6단계 수동 역전파

## 7단계 역전파 자동화

### 7.1 역전파 자동화의 시작

In [2]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

In [3]:
class Function:
    """

    * Function 클래스는 기반 클래스로서, 모든 함수에 공통되는 기능을 구현합니다.
    * 구체적인 함수는 Function 클래스를 상속한 클래스에서 구현합니다.
    """
    def __call__(self, input):
        x = input.data  # 데이터를 꺼낸다.
        y = self.forward(x)  # 구체적인 계산은 forward 메서드에서 한다.
        output = Variable(y)  # Variable 형태로 되돌린다.
        output.set_creator(self)  # 출력 변수에 창조자를 설정한다.
        self.input = input  # 입력 변수를 기억(보관)한다.
        self.output = output  # 출력도 저장한다.
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()

In [4]:
class Square(Function):
    def forward(self, x):
        return x ** 2

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

In [5]:
import numpy as np

class Exp(Function):
    def forward(self, x):
        return np.exp(x)

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

In [6]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

In [7]:
# 계산 그래프의 노드들을 거꾸로 거슬러 올라간다.
assert y.creator == C
assert y.creator.input == b
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x

### 7.2 역전파 도전!

In [8]:
y.grad = np.array(1.0)

C = y.creator  # 1. 함수를 가져온다.
b = C.input  # 2. 함수의 입력을 가져온다.
b.grad = C.backward(y.grad)  # 3. 함수의 backward 메서드를 호출한다.

In [9]:
B = b.creator  # 1. 함수를 가져온다.
a = B.input  # 2. 함수의 입력을 가져온다.
a.grad = B.backward(b.grad)  # 3. 함수의 backward 메서드를 호출한다.

In [10]:
A = a.creator  # 1. 함수를 가져온다.
x = A.input  # 2. 함수의 입력을 가져온다.
x.grad = A.backward(a.grad)  # 3. 함수의 backward 메서드를 호출한다.
print(x.grad)

3.297442541400256


### 7.3 backward 메소드 추가

In [11]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        f = self.creator  # 1. 함수를 가져온다.
        if f is not None:
            x = f.input  # 2. 함수의 입력을 가져온다.
            x.grad = f.backward(self.grad)  # 3. 함수의 backward 메서드를 호출한다.
            x.backward()  # 하나 앞 변수의 backward 메서드를 호출한다(재귀).

In [12]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# 역전파
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256
